In [1]:
!pip install transformers
!pip install seqeval

In [3]:
import numpy as np
import pickle
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')

#### METRICS ####
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score)


##### UTILS ######
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertConfig, BertModel, BertPreTrainedModel, get_linear_schedule_with_warmup, AdamW, BertTokenizerFast, BertForTokenClassification
from torch.nn import LayerNorm as BertLayerNorm

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Load the data

In [4]:
## Import Data
path = '/content/drive/My Drive/layoutlm/'
train = pickle.load(open(path + 'train.pkl', 'rb'))
val = pickle.load(open(path + 'val.pkl', 'rb'))
test = pickle.load(open(path + 'test.pkl', 'rb'))

This is simply a code to see if there are some negative bounding boxes (I find some, they were close to zero, then I set them to zero)

In [5]:
for index, elem in enumerate(train[2]):
  for i,e in enumerate(elem):
    if not all(np.array(e) >= 0):
      print(e)
      print(index, i)

[-4, 426, 213, 396]
669 0
[-3, 466, 112, 435]
669 3
[-1, 529, 223, 508]
669 6


In [6]:
train[2][669][0] = [0, 426, 213, 396]
train[2][669][3] = [0, 466, 112, 435]
train[2][669][6] = [0, 529, 223, 508]

In [7]:
for index, elem in enumerate(test[2]):
  for i,e in enumerate(elem):
    if not all(np.array(e) >= 0):
      print(e)
      print(index, i)

[-3, 891, 106, 862]
88 17


In [8]:
test[2][88][17] = [0, 891, 106, 862]

# Labels Analysis

In [9]:
all_labels = [item for sublist in train[1] for item in sublist] + [item for sublist in val[1] for item in sublist] + [item for sublist in test[1] for item in sublist]

In [10]:
from collections import Counter
Counter(all_labels)

Counter({'menu.cnt': 2429,
         'menu.discountprice': 403,
         'menu.etc': 19,
         'menu.itemsubtotal': 7,
         'menu.nm': 6599,
         'menu.num': 109,
         'menu.price': 2585,
         'menu.sub_cnt': 189,
         'menu.sub_etc': 10,
         'menu.sub_nm': 822,
         'menu.sub_price': 160,
         'menu.sub_unitprice': 14,
         'menu.unitprice': 750,
         'menu.vatyn': 9,
         'sub_total.discount_price': 191,
         'sub_total.etc': 283,
         'sub_total.othersvc_price': 6,
         'sub_total.service_price': 353,
         'sub_total.subtotal_price': 1483,
         'sub_total.tax_price': 1283,
         'total.cashprice': 1397,
         'total.changeprice': 1299,
         'total.creditcardprice': 411,
         'total.emoneyprice': 129,
         'total.menuqty_cnt': 630,
         'total.menutype_cnt': 130,
         'total.total_etc': 89,
         'total.total_price': 2120,
         'void_menu.nm': 3,
         'void_menu.price': 1})

As we can see there are some labels that contains few examples, I decided to replace them by the "neutral" label "O"

In [11]:
replacing_labels = {'menu.etc': 'O', 'mneu.itemsubtotal': 'O', 'menu.sub_etc': 'O', 'menu.sub_unitprice': 'O', 'menu.vatyn': 'O',
                  'void_menu.nm': 'O', 'void_menu.price': 'O', 'sub_total.othersvc_price': 'O'}

In [12]:
def replace_elem(elem):
  try:
    return replacing_labels[elem]
  except KeyError:
    return elem
def replace_list(ls):
  return [replace_elem(elem) for elem in ls]
train[1] = [replace_list(ls) for ls in train[1]]
val[1] = [replace_list(ls) for ls in val[1]]
test[1] = [replace_list(ls) for ls in test[1]]

In [13]:
all_labels = [item for sublist in train[1] for item in sublist] + [item for sublist in val[1] for item in sublist] + [item for sublist in test[1] for item in sublist]
Counter(all_labels)

Counter({'O': 62,
         'menu.cnt': 2429,
         'menu.discountprice': 403,
         'menu.itemsubtotal': 7,
         'menu.nm': 6599,
         'menu.num': 109,
         'menu.price': 2585,
         'menu.sub_cnt': 189,
         'menu.sub_nm': 822,
         'menu.sub_price': 160,
         'menu.unitprice': 750,
         'sub_total.discount_price': 191,
         'sub_total.etc': 283,
         'sub_total.service_price': 353,
         'sub_total.subtotal_price': 1483,
         'sub_total.tax_price': 1283,
         'total.cashprice': 1397,
         'total.changeprice': 1299,
         'total.creditcardprice': 411,
         'total.emoneyprice': 129,
         'total.menuqty_cnt': 630,
         'total.menutype_cnt': 130,
         'total.total_etc': 89,
         'total.total_price': 2120})

Now we have to save all the unique labels in a list. (mandatory for the tokenclassification)

In [14]:
labels = list(set(all_labels))

# Some aux functions

In [15]:
def set_seed(seed): ## for reproductibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

This is for making the structure of our data match the one needed by the model (mandatory)

In [16]:
class CordDataset(Dataset):
    def __init__(self, examples, tokenizer, labels, pad_token_label_id):
        features = convert_examples_to_features(
            examples,
            labels,
            max_seq_length,
            tokenizer,
            cls_token_at_end=False,
            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
            pad_on_left=False,
            # pad on the left for xlnet
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id,
        )

        self.features = features
        # Convert to Tensors and build dataset
        self.all_input_ids = torch.tensor(
            [f.input_ids for f in features], dtype=torch.long
        )
        self.all_input_mask = torch.tensor(
            [f.input_mask for f in features], dtype=torch.long
        )
        self.all_segment_ids = torch.tensor(
            [f.segment_ids for f in features], dtype=torch.long
        )
        self.all_label_ids = torch.tensor(
            [f.label_ids for f in features], dtype=torch.long
        )
        self.all_bboxes = torch.tensor([f.boxes for f in features], dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return (
            self.all_input_ids[index],
            self.all_input_mask[index],
            self.all_segment_ids[index],
            self.all_label_ids[index],
            self.all_bboxes[index],
        )

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(
        self,
        input_ids,
        input_mask,
        segment_ids,
        label_ids,
        boxes
    ):
        assert (
            0 <= all(boxes) <= 1000
        ), "Error with input bbox ({}): the coordinate value is not between 0 and 1000".format(
            boxes
        )
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.boxes = boxes

def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    cls_token_box=[0, 0, 0, 0],
    sep_token_box=[1000, 1000, 1000, 1000],
    pad_token_box=[0, 0, 0, 0],
    pad_token_segment_id=0,
    pad_token_label_id=-1,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for i in range(len(examples[0])):
        width, height = 1000, 1000
        words = examples[0]
        labels = examples[1]
        boxes = examples[2]

        tokens = []
        token_boxes = []
        label_ids = []
        for word, label, box in zip(
            words[i], labels[i], boxes[i]
        ):
            if len(word) < 1: # SKIP EMPTY WORD
              continue
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            token_boxes.extend([box] * len(word_tokens))
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend(
                [label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        token_boxes += [sep_token_box]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            token_boxes += [sep_token_box]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            token_boxes += [cls_token_box]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            token_boxes = [cls_token_box] + token_boxes
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = (
                [0 if mask_padding_with_zero else 1] * padding_length
            ) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
            token_boxes = ([pad_token_box] * padding_length) + token_boxes
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length
            token_boxes += [pad_token_box] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length
        assert len(token_boxes) == max_seq_length

        features.append(
            InputFeatures(
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                label_ids=label_ids,
                boxes=token_boxes,
            )
        )
    return features


This function is for the evaluation (loss, f1, precision , recall)

In [17]:
def results(preds, out_label_ids, labels, loss_):
  preds = np.argmax(preds, axis=2)

  label_map = {i: label for i, label in enumerate(labels)}

  out_label_list = [[] for _ in range(out_label_ids.shape[0])]
  preds_list = [[] for _ in range(out_label_ids.shape[0])]

  for i in range(out_label_ids.shape[0]):
      for j in range(out_label_ids.shape[1]):
          if out_label_ids[i, j] != pad_token_label_id:
              out_label_list[i].append(label_map[out_label_ids[i][j]])
              preds_list[i].append(label_map[preds[i][j]])

  results = {
      "loss": loss_,
      "precision": precision_score(out_label_list, preds_list),
      "recall": recall_score(out_label_list, preds_list),
      "f1": f1_score(out_label_list, preds_list),
  }
  return results

# Load models

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_path = '/content/drive/My Drive/layoutlm/'
num_labels = len(labels)
config_class, model_class, tokenizer_class = BertConfig, BertForTokenClassification, BertTokenizerFast
config = config_class.from_pretrained(model_path, num_labels=num_labels+1)
tokenizer = tokenizer_class.from_pretrained('bert-large-uncased', do_lower_case=True)
model = model_class.from_pretrained('bert-large-uncased', from_tf=bool(".ckpt" in model_path), config=config)
model.to(device)

# Generate our training / validation set

In [20]:
sum_ = []
for x in train[0]:
  sum_.append(len(x))
print(max(sum_))
print(min(sum_))
print(sum(sum_)/len(sum_))

135
5
24.21375


In [22]:
max_seq_length = 150
pad_token_label_id = CrossEntropyLoss().ignore_index
train_dataset = CordDataset(train, tokenizer, labels, pad_token_label_id)
validation_dataset = CordDataset(val, tokenizer, labels, pad_token_label_id)
model_type = 'bert'

In [23]:
train_batch_size = 8
learning_rate = 1e-4
adam_epsilon = 1e-8
weight_decay = 0.0
num_train_epochs = 4 ## To fine-tune (adding drop out so that It can lead to overfit less)
max_steps = 0
gradient_accumulation_steps = 1
max_grad_norm = 1.0
warmup_steps = 0
seed = 42

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=None,
    )
valid_sampler = RandomSampler(validation_dataset)
valid_dataloader = DataLoader(
        validation_dataset,
        sampler=valid_sampler,
        batch_size=train_batch_size,
        collate_fn=None,
    )
if max_steps > 0:
    t_total = max_steps
    num_train_epochs = (
        args.max_steps
        // (len(train_dataloader) // gradient_accumulation_steps)
        + 1
    )
else:
    t_total = (
        len(train_dataloader)
        // gradient_accumulation_steps
        * num_train_epochs
    )
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
optimizer = AdamW(
        optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon
    )
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
    )

# TRAIN

In [24]:
global_step = 0
model.zero_grad()
train_iterator = trange(int(num_train_epochs), desc="Epoch")
set_seed(seed)
for _ in train_iterator:
  #epoch_iterator = tqdm(
      #train_dataloader, desc="Iteration")
  tr_loss = 0.0
  nb_train_steps = 0
  preds_train = None
  out_label_ids = None
  for step, batch in enumerate(train_dataloader):
      model.train()
      inputs = {
          "input_ids": batch[0].to(device),
          "attention_mask": batch[1].to(device),
          "labels": batch[3].to(device),
      }
      if model_type in ["layoutlm"]:
          inputs["bbox"] = batch[4].to(device)
      inputs["token_type_ids"] = (
          batch[2].to(device) if model_type in ["bert", "layoutlm"] else None)
      outputs = model(**inputs)
      # model outputs are always tuple in pytorch-transformers (see doc)
      loss, logits = outputs
      loss.backward()

      tr_loss += loss.item()
      #if (step+1) % 25 == 0:
        #print(f"Train Epoch : {step+1}/{len(train_dataloader)}")

      if (step + 1) % gradient_accumulation_steps == 0:
          torch.nn.utils.clip_grad_norm_(
                  model.parameters(), max_grad_norm
              )
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          model.zero_grad()
          global_step += 1
      nb_train_steps += 1
      if preds_train is None:
          preds_train = logits.detach().cpu().numpy()
          out_label_ids = inputs["labels"].detach().cpu().numpy()
      else:
          preds_train = np.append(preds_train, logits.detach().cpu().numpy(), axis=0)
          out_label_ids = np.append(
              out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
  res = results(preds_train, out_label_ids, labels, tr_loss/len(train_dataloader))
  print('Train Results', res)

  ###### EVALUATION #######

  #epoch_iterator = tqdm(valid_dataloader, desc="Iteration")
  eval_loss = 0.0
  nb_eval_steps = 0
  preds_val = None
  out_label_ids = None
  model.eval()
  for step, batch in enumerate(valid_dataloader):
    with torch.no_grad():
      inputs = {
          "input_ids": batch[0].to(device),
          "attention_mask": batch[1].to(device),
          "labels": batch[3].to(device),
      }
      if model_type in ["layoutlm"]:
          inputs["bbox"] = batch[4].to(device)
      inputs["token_type_ids"] = (
          batch[2].to(device) if model_type in ["bert", "layoutlm"] else None)
      # model outputs are always tuple in pytorch-transformers (see doc)
      outputs = model(**inputs)
      tmp_eval_loss, logits = outputs[:2]
      eval_loss += tmp_eval_loss.item()
    nb_eval_steps += 1
    if preds_val is None:
      preds_val = logits.detach().cpu().numpy()
      out_label_ids = inputs["labels"].detach().cpu().numpy()
    else:
      preds_val = np.append(preds_val, logits.detach().cpu().numpy(), axis=0)
      out_label_ids = np.append(
          out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0
      )
  eval_loss = eval_loss / nb_eval_steps
  res = results(preds_val, out_label_ids, labels, eval_loss)
  print('Validation results',res)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train Results {'loss': 0.7824301970750094, 'precision': 0.6843320017754105, 'recall': 0.7189889945905614, 'f1': 0.7012325465047529}


Epoch:  25%|██▌       | 1/4 [02:39<07:58, 159.63s/it]

Validation results {'loss': 0.25740169705106664, 'precision': 0.893312101910828, 'recall': 0.908502024291498, 'f1': 0.9008430349257326}
Train Results {'loss': 0.23918551264330745, 'precision': 0.8841217455078841, 'recall': 0.8994590561462413, 'f1': 0.8917244567730005}


Epoch:  50%|█████     | 2/4 [05:18<05:18, 159.41s/it]

Validation results {'loss': 0.20990515396428797, 'precision': 0.9218877135882831, 'recall': 0.9174089068825911, 'f1': 0.9196428571428572}
Train Results {'loss': 0.1167666693078354, 'precision': 0.9442747384985652, 'recall': 0.9514083193434061, 'f1': 0.9478281068524972}


Epoch:  75%|███████▌  | 3/4 [07:57<02:39, 159.26s/it]

Validation results {'loss': 0.19350521438396895, 'precision': 0.9245585874799358, 'recall': 0.9327935222672065, 'f1': 0.9286577992744861}
Train Results {'loss': 0.060243648015894, 'precision': 0.971986970684039, 'recall': 0.9740720014922589, 'f1': 0.973028369124703}


Epoch: 100%|██████████| 4/4 [10:35<00:00, 158.91s/it]

Validation results {'loss': 0.18163511672845253, 'precision': 0.946645109135004, 'recall': 0.9481781376518219, 'f1': 0.9474110032362459}


# TEST

In [25]:
for index, e in enumerate(test[2]):
  for index_,l in enumerate(e):
    if l[1] < l[3]:
      print(index, index_)
      print(l)

54 15
[105, 726, 498, 738]


In [26]:
test[2][54][15] = [105, 738, 498, 726]

In [27]:
test_dataset = CordDataset(test, tokenizer, labels, pad_token_label_id)

In [28]:
def results_test(preds, out_label_ids, labels):
  preds = np.argmax(preds, axis=2)

  label_map = {i: label for i, label in enumerate(labels)}

  out_label_list = [[] for _ in range(out_label_ids.shape[0])]
  preds_list = [[] for _ in range(out_label_ids.shape[0])]

  for i in range(out_label_ids.shape[0]):
      for j in range(out_label_ids.shape[1]):
          if out_label_ids[i, j] != pad_token_label_id:
              out_label_list[i].append(label_map[out_label_ids[i][j]])
              preds_list[i].append(label_map[preds[i][j]])

  results = {
      "precision": precision_score(out_label_list, preds_list),
      "recall": recall_score(out_label_list, preds_list),
      "f1": f1_score(out_label_list, preds_list),
  }
  return results, classification_report(out_label_list, preds_list)

In [29]:
val_r, class_r = results_test(preds_val, out_label_ids, labels)

In [31]:
val_r

{'f1': 0.9474110032362459,
 'precision': 0.946645109135004,
 'recall': 0.9481781376518219}

In [30]:
print(class_r)

                         precision    recall  f1-score   support

                enu.cnt       0.99      0.97      0.98       213
      enu.discountprice       0.50      0.50      0.50         4
                 enu.nm       0.96      0.97      0.97       221
                enu.num       0.50      0.50      0.50         4
              enu.price       0.96      0.97      0.97       222
            enu.sub_cnt       0.88      0.94      0.91        32
             enu.sub_nm       0.91      0.96      0.93        45
          enu.sub_price       0.77      0.67      0.71        15
          enu.unitprice       0.98      0.94      0.96        52
         otal.cashprice       0.96      0.97      0.96        66
       otal.changeprice       0.98      0.95      0.97        66
   otal.creditcardprice       0.78      0.93      0.85        15
       otal.emoneyprice       1.00      1.00      1.00         4
       otal.menuqty_cnt       0.88      0.88      0.88        26
      otal.menutype_cnt 

In [32]:
test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(
        test_dataset,
        sampler=test_sampler,
        batch_size=2,
        collate_fn=None,
    )
nb_eval_steps = 0
preds_test = None
out_label_ids = None
model.eval()
for step, batch in enumerate(train_dataloader):
  with torch.no_grad():
    inputs = {
        "input_ids": batch[0].to(device),
        "attention_mask": batch[1].to(device),
        "labels": batch[3].to(device),
    }
    if model_type in ["layoutlm"]:
        inputs["bbox"] = batch[4].to(device)
    inputs["token_type_ids"] = (
        batch[2].to(device) if model_type in ["bert", "layoutlm"] else None)
    # model outputs are always tuple in pytorch-transformers (see doc)
    outputs = model(**inputs)
    _, logits = outputs[:2]
  if preds_test is None:
    preds_test = logits.detach().cpu().numpy()
    out_label_ids = inputs["labels"].detach().cpu().numpy()
  else:
    preds_test = np.append(preds_test, logits.detach().cpu().numpy(), axis=0)
    out_label_ids = np.append(
        out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0
    )

res, report = results_test(preds_test, out_label_ids, labels)

In [33]:
res

{'f1': 0.985912865006064,
 'precision': 0.9861888764464353,
 'recall': 0.9856370080208916}

In [34]:
print(report)

                         precision    recall  f1-score   support

                enu.cnt       1.00      1.00      1.00      1887
      enu.discountprice       0.84      0.93      0.88        82
       enu.itemsubtotal       0.00      0.00      0.00         1
                 enu.nm       0.99      0.99      0.99      2060
                enu.num       1.00      0.88      0.93        90
              enu.price       0.99      0.99      0.99      2058
            enu.sub_cnt       0.98      0.98      0.98       140
             enu.sub_nm       0.96      0.99      0.97       273
          enu.sub_price       0.96      0.94      0.95       121
          enu.unitprice       0.99      0.98      0.99       612
         otal.cashprice       0.99      0.99      0.99       527
       otal.changeprice       0.99      0.98      0.99       505
   otal.creditcardprice       0.97      0.97      0.97       119
       otal.emoneyprice       0.96      0.96      0.96        50
       otal.menuqty_cnt 